#Face detection with `skimage`

In this lesson, we will see how to use `skimage` tools to locate human faces in an image. The techniques below might be used as the first step in facial *recognition*; before we can try to determine who appears in an image, based on their faces, we need to *detect* the areas in the image that are faces. 

In the code that follows, we will use a pre-trained <a href="https://en.wikipedia.org/wiki/Cascading_classifiers">cascade classifier</a>, a specific type of machine learning algorithm designed to quickly detect a given class of objects (faces, in our case) in an image. *Pre-trained* means that researchers have already developed the algorithm and have "taught" it how to detect objects by giving it many images that are positive and negative instances -- images that are faces and images that are not faces. All we have to do is load up the trained classifier and learn to use it.

We begin, as ususal, with some standard imports and configuration to allow us to view images in Colab.

In [ ]:
# one-time imports and configuration
import skimage.io

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

from matplotlib import pyplot as plt


Next, we have some imports specifically related to using our cascade classifier, as shown in the following cell:

In [ ]:
# the data module has several sample images to practice with
from skimage import data
# the feature.Cascade class encapsulates a cascade classifier
from skimage.feature import Cascade
# the patches matplotlib module will allow us to easily highlight faces
from matplotlib import patches

Now, we can create our cascade classifier. The trained face detection classifier is provided as part of the `skimage` distribution. In order to use it, we need to determine the location and filename of the classifier configuration and parameters, which is stored in an `.xml` file somewhere on our computer. Since that location may vary considerably from installation to installation, `skimage` provides a function to return the full path to the `.xml` file, 

```
data.lbp_frontal_face_cascade_filename()
```

On the author's Colab setup, a call to that function returns this path, as a Python string:

```
/usr/local/lib/python3.7/dist-packages/skimage/data/lbpcascade_frontalface_opencv.xml
```

The next cell calls the function and saves the classifier's file name in a variable named `classifierFileName`.

In [ ]:
# Get the file name of the trained classifier description
classifierFileName = data.lbp_frontal_face_cascade_filename()

Next, we create an instance of the `Cascasde` class, using the classifier parameters contained in the initiation file we found in the previous step. This is accomplised by passing the file name of the classifier description to the `Cascade` constructor, as shown in the next cell.

In [ ]:
# Initialize the detector cascade.
detector = Cascade(classifierFileName)

Everything to this point we only need to do once. In other words, now that we have created our instace of the `Cascade` class, `detector`, we can use the instance to detect faces in any number of different images. 

So, let's start detecting faces! We will begin by trying to detect the face in one of the sample images in the `skimage.data` module. In particular, we will use an image of <a href="https://en.wikipedia.org/wiki/Eileen_Collins">Eileen Collins</a>, a former NASA astronaut. We do this via the `data.astronaut()` function call. The function returns the image in a `numpy` array, just like the images we have been using all along. The array is a 3D array with RGB values in the range [0, 255].

The next cell loads the image and displays it.

In [ ]:
img = data.astronaut()
skimage.io.imshow(img)
plt.show()

Now that we have an image, we can use our trained classifier to detect the face(s) in the image. Assuming that our classifier object is named `detector`, we do this with a call to the `detector.detect_multi_scale()` method. The `multi_scale` portion of the method name means that the classifier will try to find faces of a variety of sizes in the image. 

This function examines all of the areas of a certain minimum size in the image, passes them through the classifier, and decides if each represents a face or not. Then, to detect faces that might be larger, the process is repeated for areas of a larger size. This whole process continues until some specified maximum size is reached. 

There are five parameters we have to provide, as follows. The parameters are positional, but we will provide names for the last four to make the code easier to read. 

First, we provide the `numpy` array representing our image. Here, we have named the array `img`. 

Next, we provide the `scale_factor`. This is a floating point number that dictates how quickly the search areas grow. For example, a value of 2 here would make each successive search area twice as big as the previous one. Values closer to 1 will make the classifier more accurate, but slower, while larger values will make the classifier faster but less accurate. 

Third is the `step_ratio` parameter. This is a floating point number that represents the step size between each search area. Using a value of 1 here will look at *all* of the possible positions for each search area in the image, resulting in a slow but more accurate search. Increasing this value puts more pixels between each successive search area, resulting in a faster but less accurate search. According to the <a href="https://scikit-image.org/docs/dev/api/skimage.feature.html?highlight=template#skimage.feature.Cascade">`Cascasde` documentation</a>, values in the range [1, 1.5] provide good results. 

The next two parameters, `min_size` and `max_size`, are both tuples, each containing two integers. These are the minimum size and maximum size of the search areas. If we have a rough guess of how large the faces in our image are likely to be, we should bracket that assumed size between `min_size` and `max_size`, so the classifier does not spend too much time looking at areas that are either to small or too large to be actual faces in the image. 

The next cell shows usage of the `detect_multi_scale()` method call for the sample image. Examination of the image tells us that Col. Collins' face is approximately 100x100 pixels, so we use `min_size = (60, 60)` and `max_size = (120, 120)`.

In [ ]:
# use our classifier to detect faces in the image
detected = detector.detect_multi_scale(img, 
                                       scale_factor = 1.1, 
                                       step_ratio = 1, 
                                       min_size = (60, 60), 
                                       max_size = (120, 120))



The `detect_multi_scale()` method returns a list of dictionaries, where each dictionary has information about one of the faces detected in the image. Each dictionary has four keys:

- `'r'` : row of the top-left corner of the face
- `'c'` : column of the top-left corner of the face
- `'width'` : width of the face
- `'height'` : height of the face

In other words, the list of dictionaries describes rectangles that surround each of the faces detected by the classifier. 

We can use that information in a variety of ways. For instance, we could now use slicing to extract sub-images that represente just the faces, in preparation for using other machine learning tools to recognize the faces.

Here, we will simply use the list of dictionaries to draw a colored rectangle for each face on top of the original image. The next cell shows one way to accomplish this.

In [ ]:
# put the original image in the plot area
plt.imshow(img)

# get the plot area's drawing surface
img_desc = plt.gca()

# for each of the detected faces...
for face in detected:
    # draw a rectangle showing the face's location
    img_desc.add_patch(
        patches.Rectangle(
            (face['c'], face['r']),
            face['width'],
            face['height'],
            fill=False,
            color='r',
            linewidth=2
        )
    )

# display the annoated image
plt.show()

> ---
> **Dectect faces in your own image.**
> 
> Now it is time for you to try your hand at face 
> detection! Find an image of your own choosing that 
> contains multiple faces, like a group photo, for 
> instance. Load that image, then apply the 
> `detector.detect_multi_scale()` method to try to 
> find the faces in the image. 
> 
> This detector seems to be sensitive to the 
> `scale_factor`, `min_size`, and `max_size` 
> parameters, so you will probably have to experiment
> with these values in order to detect as many faces 
> as possible.
> 
> ---

In [ ]:
# TODO: load your own image into the img variable


In [ ]:
# TODO: use detector.detect_multi_scale() to find the faces
# use our classifier to detect faces in the image


In [ ]:
# TODO: execute this cell to show highlight the faces
# put the original image in the plot area
plt.imshow(img)

# get the plot area's drawing surface
img_desc = plt.gca()

# for each of the detected faces...
for face in detected:
    # draw a rectangle showing the face's location
    img_desc.add_patch(
        patches.Rectangle(
            (face['c'], face['r']),
            face['width'],
            face['height'],
            fill=False,
            color='r',
            linewidth=2
        )
    )

# display the annoated image
plt.show()